<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Langgraph_Agentic_RAG_Cyber_AI_Copilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cyber AI Copilot for Security and Intelligence Domain**

In [17]:
!pip install --upgrade --quiet sentence-transformers langchain langchain-groq langchain-pinecone langchain-community langchain-core duckduckgo-search langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.5 MB/s eta 0:00:00


In [18]:
import os
from typing import List, Dict, Any, Optional, TypedDict
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END
from langchain_community.tools import DuckDuckGoSearchResults
from dotenv import load_dotenv

GROQ_API_KEY = "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"
PINECONE_API_KEY = "8e15b925-3b96-497d-b20a-08d308782b83"
PINECONE_ENVIRONMENT = "us-east-1"

In [19]:
# Initialize the LLM and embeddings
llm = ChatGroq(temperature=0, model="llama3-8b-8192", api_key=GROQ_API_KEY)
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# Debug statement to check if embeddings are initialized
if embeddings is None:
    raise ValueError("Embeddings model failed to initialize")

pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone_index = pc.Index("new-cyber-search")
vector_store = PineconeVectorStore(index=pinecone_index, embedding=embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
# Define the state
class AgentState(TypedDict):
    messages: List[Dict[str, str]]
    memory: Optional[Dict[str, Any]]

# Define the tools
search_tool = DuckDuckGoSearchResults()

def vector_search(query: str) -> str:
    results = vector_store.similarity_search(query, k=3)
    return "\n".join([doc.page_content for doc in results])

# Define the nodes
def search_duck(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    result = search_tool.run(query)
    state["messages"].append({"role": "tool", "content": f"DuckDuckGo Search Result: {result}"})
    return state

def search_vector(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    result = vector_search(query)
    state["messages"].append({"role": "tool", "content": f"Vector Search Result: {result}"})
    return state

In [21]:
def generate_response(state: AgentState) -> AgentState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an advanced AI copilot specializing in cybersecurity and intelligence. Your role is to provide accurate, up-to-date information and insights on cyber threats, vulnerabilities, and intelligence matters. Use the following guidelines:

1. Analyze the user's query and the provided search results thoroughly.
2. Prioritize the most relevant and recent information from both DuckDuckGo and vector search results.
3. Provide a comprehensive response that addresses the user's query, incorporating insights from both search sources.
4. If there are conflicting pieces of information, acknowledge them and provide a balanced view.
5. Always consider the ethical implications of the information you're providing, especially in sensitive cybersecurity matters.
6. If asked about specific attack techniques or vulnerabilities, provide information for defensive purposes only.
7. Recommend reliable sources for further reading when appropriate.
8. If you're unsure about any information, clearly state that and suggest ways to verify or obtain more accurate data.

Remember, you're assisting in a professional capacity in the cybersecurity and intelligence domain. Maintain a tone that is informative, analytical, and security-conscious."""),
        ("human", "{input}"),
        ("human", "DuckDuckGo Search Result: {duck_result}"),
        ("human", "Vector Search Result: {vector_result}"),
        ("human", "Based on the above information, provide a comprehensive response to the query."),
    ])

    chain = LLMChain(llm=llm, prompt=prompt)

    response = chain.run(
        input=state["messages"][-1]["content"],
        duck_result=next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "DuckDuckGo" in m["content"]), "No DuckDuckGo results."),
        vector_result=next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "Vector Search" in m["content"]), "No vector search results.")
    )

    state["messages"].append({"role": "assistant", "content": response})
    return state

# Define the graph
workflow = StateGraph(AgentState)

# Define the nodes
workflow.add_node("search_duck", search_duck)
workflow.add_node("search_vector", search_vector)
workflow.add_node("generate_response", generate_response)

# Define the edges
workflow.add_edge("search_duck", "search_vector")
workflow.add_edge("search_vector", "generate_response")
workflow.add_edge("generate_response", END)

# Set the entry point
workflow.set_entry_point("search_duck")

# Compile the graph
graph = workflow.compile()

In [22]:
# Function to run the agent
def run_agent(query: str, memory: Optional[Dict[str, Any]] = None) -> AgentState:
    state = AgentState(messages=[{"role": "human", "content": query}], memory=memory)
    result = graph.invoke(state)
    return result

# Example usage
if __name__ == "__main__":
    query = "What are the latest trends in ransomware attacks in India last 3 Months?"
    result = run_agent(query)
    for message in result["messages"]:
        if message["role"] == "assistant":
            print("AI Copilot:", message["content"])

<ipython-input-21-453ec718a411>:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
<ipython-input-21-453ec718a411>:23: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(


AI Copilot: The query appears to be related to ransomware attacks and their trends in 2024. Based on the provided Vector Search Result and DuckDuckGo Search Result, here is a comprehensive response:

Ransomware attacks are expected to continue growing in 2024, with cybercriminals becoming more sophisticated in their methods and expanding their reach. According to ReliaQuest, there has been a 35% increase in ransomware victims from the professional, scientific, and technical services (PSTS) sector, which includes software companies.

The 2024 Thales Data Threat Report reveals that 93% of IT professionals believe security threats are increasing in volume or severity, a significant rise from 47% in the previous year. The report also found that 11% of respondents in India experienced ransomware attacks in the past year, despite only 20% having a formal ransomware plan in place.

The report also highlights the growing threat of ransomware attacks in India, with the country ranking 4th globa